<a href="https://colab.research.google.com/github/royapakzad/Booklet/blob/master/GPT_FB_MSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import json
import random
import re

In [0]:
!git clone -b small https://github.com/Tenoke/gpt-2.git

In [0]:
cd gpt-2

In [0]:
!pip3 install -r requirements.txt

In [0]:
!sh download_model.sh 117M

In [0]:
!curl 'https://bigzipfiles.facebook.com/p/dl/download/file.php?r=100000045203418&t=100000045203418&j=11&i=2423522277659255&ext=1552347802&hash=AaDXnDMervVmbTPV' -H 'authority: bigzipfiles.facebook.com' -H 'upgrade-insecure-requests: 1' -H 'dnt: 1' -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'referer: https://www.facebook.com/' -H 'accept-encoding: gzip, deflate, br' -H 'accept-language: en-GB,en;q=0.9,bg-BG;q=0.8,bg;q=0.7,en-US;q=0.6' -H 'cookie: datr=qi5tXEI5YtkypxAdISqJGNZd; sb=qi5tXJPGdhJhbt5N04dUGcUv; c_user=100000045203418; xs=29%3AgDb81L_oJkC4Kg%3A2%3A1550659246%3A4402%3A5224; pl=n; dpr=0.8999999761581421; spin=r.1000481805_b.trunk_t.1552342837_s.1_v.2_; fr=1BXrWEDVIeChp7NLC.AWXvyWCWdyCZAdTM2bSw40pj8-E.BcbS6q.Hm.AAA.0.0.BchuAL.AWWf6I2i; presence=EDvF3EtimeF1552346587EuserFA21BB45203418A2EstateFDsb2F1552344421952EatF1552345159684Et3F_5bDiFA2user_3a1B00305017170A2EoF1EfF1CAcDiFA2user_3a1B01308116906A2EoF2EfF3CAcDiFA2user_3a1B08124067081A2EoF3EfF2C_5dEutc3F1552345156737G552346587474CEchFDp_5f1BB45203418F1CC; act=1552346589369%2F2; wd=901x697; pnl_data2=eyJhIjoib25hZnRlcmxvYWQiLCJjIjoiWERZSVJlcXVlc3RDb250cm9sbGVyIiwiYiI6ZmFsc2UsImQiOiIvcC9kbC9kb3dubG9hZC9maWxlLnBocCIsImUiOltdfQ%3D%3D' --compressed --output fb-json.zip

In [0]:
!unzip fb-json.zip

In [0]:
files = []
for p, d, f in os.walk('messages/inbox'):
    for file in f:
        if file.endswith('message.json'):
            files.append(f'{p}/{file}')

len(files)

In [0]:
def fix_encoding(s):
  return re.sub('[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),s)

def find_cyrilic(s):
    return len(re.findall('(?i)[А-ЯЁ]', s)) > 0

def test_mostly_cyrilic(messages):
  i = 0
  check_n = min(250, len(messages))
  for msg in random.sample(messages, check_n):
    try:
      i +=find_cyrilic(fix_encoding(msg['content'])) or find_cyrilic(fix_encoding(msg['sender_name']))
    except KeyError:
      check_n -=1
  return i > check_n/5

In [0]:

text_corpus = ''
banned_names = ('vladislav',)
for file in files:
  with open(file, 'r') as f:
      try:
        msgs = json.load(f)['messages']
        msgs.reverse()
      except:
        pass
      else:
        if not test_mostly_cyrilic(msgs) and not any(bn in file for bn in banned_names):
          for msg in msgs:
            try:
              content = fix_encoding(msg['content'])
              to_add  = f"({msg['timestamp_ms']}) {msg['sender_name']}: {content}\n"
              if not find_cyrilic(to_add):
                text_corpus += to_add
            except KeyError:
              pass
          print(file)

          text_corpus += '\n\n'

 

In [0]:
with open('fb-cleaned.txt', 'w') as f:
  f.write(text_corpus)

In [0]:
!du -h fb-cleaned.txt

In [0]:
!PYTHONPATH=src ./encode.py --in-text fb-cleaned.txt --out-npz fb-cleaned.txt.npz

In [0]:
!PYTHONPATH=src ./train.py --dataset fb-cleaned.txt.npz --sample_every=250 --learning_rate=0.0001 --stop_after=251


In [0]:
!PYTHONPATH=src ./train.py --dataset fb-cleaned.txt.npz --sample_every=250 --learning_rate=0.001 --stop_after=751


In [0]:
!PYTHONPATH=src ./train.py --dataset fb-cleaned.txt.npz --sample_every=250 --learning_rate=0.0001 --beta=0.95 --stop_after=1251

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.9

In [0]:
def create_specific_file(person, files=files):
  text_corpus = ''
  for file in files:
    if person in file:
      print(file)
      with open(file, 'r') as f:
          try:
            msgs = json.load(f)['messages']
            msgs.reverse()
          except:
            pass
          else:
            for msg in msgs:
              try:
                content = fix_encoding(msg['content'])
                to_add  = f"({msg['timestamp_ms']}) {msg['sender_name']}: {content}\n"
                if not find_cyrilic(to_add):
                  text_corpus += to_add
              except KeyError:
                pass

              text_corpus += '\n\n'
      with open(f'fb-cleaned-{person}.txt', 'w') as f:
        f.write(text_corpus)
        return
